# Scraping Match Data for All Teams for 2022-23 Season of the English Premier League (FBref.com)

In [7]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

Collecting data for each time by visiting the individual team page through links from the League Table

In [13]:
standings_url = "https://fbref.com/en/comps/9/Premier-League-Stats"

In [47]:
data = requests.get(standings_url)
soup = BeautifulSoup(data.text)

In [15]:
standings_table = soup.select('table.stats_table')[0]

In [16]:
links = standings_table.find_all('a')

In [17]:
links = [l.get("href") for l in links]

In [18]:
links = [l for l in links if '/squads/' in l]
links

['/en/squads/b8fd03ef/Manchester-City-Stats',
 '/en/squads/18bb7c10/Arsenal-Stats',
 '/en/squads/b2b47a98/Newcastle-United-Stats',
 '/en/squads/19538871/Manchester-United-Stats',
 '/en/squads/822bd0ba/Liverpool-Stats',
 '/en/squads/d07537b9/Brighton-and-Hove-Albion-Stats',
 '/en/squads/361ca564/Tottenham-Hotspur-Stats',
 '/en/squads/8602292d/Aston-Villa-Stats',
 '/en/squads/cd051869/Brentford-Stats',
 '/en/squads/fd962109/Fulham-Stats',
 '/en/squads/cff3d9bb/Chelsea-Stats',
 '/en/squads/47c64c55/Crystal-Palace-Stats',
 '/en/squads/8cec06e1/Wolverhampton-Wanderers-Stats',
 '/en/squads/4ba7cbea/Bournemouth-Stats',
 '/en/squads/7c21e445/West-Ham-United-Stats',
 '/en/squads/e4a775cb/Nottingham-Forest-Stats',
 '/en/squads/d3fd31cc/Everton-Stats',
 '/en/squads/5bfb9659/Leeds-United-Stats',
 '/en/squads/a2d435b3/Leicester-City-Stats',
 '/en/squads/33c895d4/Southampton-Stats']

In [ ]:
team_urls = [f"https://fbref.com{l}" for l in links]

In [ ]:
team_url = team_urls[0]

In [ ]:
data = requests.get(team_url)

In [ ]:
matches = pd.read_html(data.text, match="Scores & Fixtures")

In [ ]:
matches[0].head()

In [ ]:
soup = BeautifulSoup(data.text)
links = soup.find_all('a')
links = [l.get("href") for l in links]
links = [l for l in links if l and '/all_comps/shooting/' in l]
links

In [ ]:
data = requests.get(f"https://fbref.com{links[0]}")

Combining the shooting data with the previously scraped data.

In [ ]:
shooting = pd.read_html(data.text, match="Shooting")[0]

In [ ]:
shooting.head()

In [ ]:
shooting.columns = shooting.columns.droplevel()

In [ ]:
team_data = matches[0].merge(shooting[["Date", "Sh", "SoT", "Dist", "PK", "PKatt", "npxG"]], on="Date")

In [ ]:
team_data.head()

### Repeating the process for the all the teams

In [19]:
all_matches = []

In [20]:
standings_url = 'https://fbref.com/en/comps/9/Premier-League-Stats'

In [21]:
import time

data = requests.get(standings_url)
soup = BeautifulSoup(data.text)
standings_table = soup.select('table.stats_table')[0]

links = [l.get("href") for l in standings_table.find_all('a')]
links = [l for l in links if '/squads/' in l]
team_urls = [f"https://fbref.com{l}" for l in links]

for team_url in team_urls:
    team_name = team_url.split('/')[-1].replace("-Stats", "")

    time.sleep(3)
    data = requests.get(team_url)
    matches = pd.read_html(data.text, match="Scores & Fixtures")[0]
    soup = BeautifulSoup(data.text)
    links = [l.get("href") for l in soup.find_all('a')]
    links = [l for l in links if l and 'all_comps/shooting/' in l]
    time.sleep(5)
    data = requests.get(f"https://fbref.com{links[0]}")
    shooting = pd.read_html(data.text, match="Shooting")[0]
    shooting.columns = shooting.columns.droplevel()

    try:
        team_data = matches.merge(shooting[["Date", "Sh", "SoT", "Dist", "PK", "PKatt", "npxG"]], on="Date")
    except ValueError:
        continue

    team_data = team_data[team_data["Comp"] == "Premier League"]
    team_data["Season"] = year
    team_data["Club"] = team_name
    all_matches.append(team_data)
    time.sleep(6)

In [22]:
match_df = pd.concat(all_matches)

In [23]:
match_df

,Date,Time,Comp,Round,Day,Venue,Result,GF,GA,Opponent,...,Match Report,Notes,Sh,SoT,Dist,PK,PKatt,npxG,Season,Club
1,2022-08-07,16:30,Premier League,Matchweek 1,Sun,Away,W,2.0,0.0,West Ham,...,Match Report,NaN,13.0,1.0,18.7,1,1,1.4,2022,Manchester-City
2,2022-08-13,15:00,Premier League,Matchweek 2,Sat,Home,W,4.0,0.0,Bournemouth,...,Match Report,NaN,19.0,7.0,17.5,0,0,1.7,2022,Manchester-City
3,2022-08-21,16:30,Premier League,Matchweek 3,Sun,Away,D,3.0,3.0,Newcastle Utd,...,Match Report,NaN,21.0,10.0,16.2,0,0,2.1,2022,Manchester-City
4,2022-08-27,15:00,Premier League,Matchweek 4,Sat,Home,W,4.0,2.0,Crystal Palace,...,Match Report,NaN,18.0,5.0,14.1,0,0,2.2,2022,Manchester-City
5,2022-08-31,19:30,Premier League,Matchweek 5,Wed,Home,W,6.0,0.0,Nott'ham Forest,...,Match Report,NaN,17.0,9.0,14.8,0,0,3.3,2022,Manchester-City
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40,2023-04-21,20:00,Premier League,Matchweek 32,Fri,Away,D,3,3,Arsenal,...,Match Report,NaN,8.0,6.0,12.4,0,0,1.3,2022,Southampton
41,2023-04-27,19:45,Premier League,Matchweek 33,Thu,Home,L,0,1,Bournemouth,...,Match Report,NaN,11.0,2.0,19.9,0,0,0.5,2022,Southampton
42,2023-04-30,14:00,Premier League,Matchweek 34,Sun,Away,L,1,3,Newcastle Utd,...,Match Report,NaN,4.0,3.0,17.3,0,0,0.8,2022,Southampton
43,2023-05-08,20:00,Premier League,Matchweek 35,Mon,Away,L,3,4,Nott'ham Forest,...,Match Report,NaN,18.0,4.0,14.0,1,1,2.4,2022,Southampton


Exporting the scraped data to a csv file

In [24]:
match_df.to_csv("matches22-23.csv")

In [25]:
match_df["Season"] = 2023

In [26]:
match_df.head()

,Date,Time,Comp,Round,Day,Venue,Result,GF,GA,Opponent,...,Match Report,Notes,Sh,SoT,Dist,PK,PKatt,npxG,Season,Club
1,2022-08-07,16:30,Premier League,Matchweek 1,Sun,Away,W,2.0,0.0,West Ham,...,Match Report,NaN,13.0,1.0,18.7,1,1,1.4,2023,Manchester-City
2,2022-08-13,15:00,Premier League,Matchweek 2,Sat,Home,W,4.0,0.0,Bournemouth,...,Match Report,NaN,19.0,7.0,17.5,0,0,1.7,2023,Manchester-City
3,2022-08-21,16:30,Premier League,Matchweek 3,Sun,Away,D,3.0,3.0,Newcastle Utd,...,Match Report,NaN,21.0,10.0,16.2,0,0,2.1,2023,Manchester-City
4,2022-08-27,15:00,Premier League,Matchweek 4,Sat,Home,W,4.0,2.0,Crystal Palace,...,Match Report,NaN,18.0,5.0,14.1,0,0,2.2,2023,Manchester-City
5,2022-08-31,19:30,Premier League,Matchweek 5,Wed,Home,W,6.0,0.0,Nott'ham Forest,...,Match Report,NaN,17.0,9.0,14.8,0,0,3.3,2023,Manchester-City


### Data cleaning and combining with existing old data

In [29]:
match_df.columns = [c.lower() for c in match_df.columns]

In [39]:
match_df.drop(['npxg'], axis = 1, inplace=True)

In [27]:
match_df_2 = pd.read_csv("matches.csv")

In [40]:
match_df_2.rename(columns = {'team':'club'}, inplace = True)

In [41]:
match_df_2.head()

,date,time,comp,round,day,venue,result,gf,ga,opponent,...,referee,match report,notes,sh,sot,dist,pk,pkatt,season,club
0,2021-08-15,16:30,Premier League,Matchweek 1,Sun,Away,L,0.0,1.0,Tottenham,...,Anthony Taylor,Match Report,NaN,18.0,4.0,16.9,0.0,0.0,2022,Manchester City
1,2021-08-21,15:00,Premier League,Matchweek 2,Sat,Home,W,5.0,0.0,Norwich City,...,Graham Scott,Match Report,NaN,16.0,4.0,17.3,0.0,0.0,2022,Manchester City
2,2021-08-28,12:30,Premier League,Matchweek 3,Sat,Home,W,5.0,0.0,Arsenal,...,Martin Atkinson,Match Report,NaN,25.0,10.0,14.3,0.0,0.0,2022,Manchester City
3,2021-09-11,15:00,Premier League,Matchweek 4,Sat,Away,W,1.0,0.0,Leicester City,...,Paul Tierney,Match Report,NaN,25.0,8.0,14.0,0.0,0.0,2022,Manchester City
4,2021-09-18,15:00,Premier League,Matchweek 5,Sat,Home,D,0.0,0.0,Southampton,...,Jonathan Moss,Match Report,NaN,16.0,1.0,15.7,0.0,0.0,2022,Manchester City


In [34]:
match_df_2.drop(match_df_2.columns[[0]], axis = 1, inplace=True)

In [38]:
match_df_2.drop(['fk'], axis = 1, inplace=True)

In [42]:
match_df.head()

,date,time,comp,round,day,venue,result,gf,ga,opponent,...,referee,match report,notes,sh,sot,dist,pk,pkatt,season,club
1,2022-08-07,16:30,Premier League,Matchweek 1,Sun,Away,W,2.0,0.0,West Ham,...,Michael Oliver,Match Report,NaN,13.0,1.0,18.7,1,1,2023,Manchester-City
2,2022-08-13,15:00,Premier League,Matchweek 2,Sat,Home,W,4.0,0.0,Bournemouth,...,David Coote,Match Report,NaN,19.0,7.0,17.5,0,0,2023,Manchester-City
3,2022-08-21,16:30,Premier League,Matchweek 3,Sun,Away,D,3.0,3.0,Newcastle Utd,...,Jarred Gillett,Match Report,NaN,21.0,10.0,16.2,0,0,2023,Manchester-City
4,2022-08-27,15:00,Premier League,Matchweek 4,Sat,Home,W,4.0,2.0,Crystal Palace,...,Darren England,Match Report,NaN,18.0,5.0,14.1,0,0,2023,Manchester-City
5,2022-08-31,19:30,Premier League,Matchweek 5,Wed,Home,W,6.0,0.0,Nott'ham Forest,...,Paul Tierney,Match Report,NaN,17.0,9.0,14.8,0,0,2023,Manchester-City


In [43]:
match_df_2.head()

,date,time,comp,round,day,venue,result,gf,ga,opponent,...,referee,match report,notes,sh,sot,dist,pk,pkatt,season,club
0,2021-08-15,16:30,Premier League,Matchweek 1,Sun,Away,L,0.0,1.0,Tottenham,...,Anthony Taylor,Match Report,NaN,18.0,4.0,16.9,0.0,0.0,2022,Manchester City
1,2021-08-21,15:00,Premier League,Matchweek 2,Sat,Home,W,5.0,0.0,Norwich City,...,Graham Scott,Match Report,NaN,16.0,4.0,17.3,0.0,0.0,2022,Manchester City
2,2021-08-28,12:30,Premier League,Matchweek 3,Sat,Home,W,5.0,0.0,Arsenal,...,Martin Atkinson,Match Report,NaN,25.0,10.0,14.3,0.0,0.0,2022,Manchester City
3,2021-09-11,15:00,Premier League,Matchweek 4,Sat,Away,W,1.0,0.0,Leicester City,...,Paul Tierney,Match Report,NaN,25.0,8.0,14.0,0.0,0.0,2022,Manchester City
4,2021-09-18,15:00,Premier League,Matchweek 5,Sat,Home,D,0.0,0.0,Southampton,...,Jonathan Moss,Match Report,NaN,16.0,1.0,15.7,0.0,0.0,2022,Manchester City


In [45]:
df = pd.concat([match_df, match_df_2])
df

,date,time,comp,round,day,venue,result,gf,ga,opponent,...,referee,match report,notes,sh,sot,dist,pk,pkatt,season,club
1,2022-08-07,16:30,Premier League,Matchweek 1,Sun,Away,W,2.0,0.0,West Ham,...,Michael Oliver,Match Report,NaN,13.0,1.0,18.7,1.0,1.0,2023,Manchester-City
2,2022-08-13,15:00,Premier League,Matchweek 2,Sat,Home,W,4.0,0.0,Bournemouth,...,David Coote,Match Report,NaN,19.0,7.0,17.5,0.0,0.0,2023,Manchester-City
3,2022-08-21,16:30,Premier League,Matchweek 3,Sun,Away,D,3.0,3.0,Newcastle Utd,...,Jarred Gillett,Match Report,NaN,21.0,10.0,16.2,0.0,0.0,2023,Manchester-City
4,2022-08-27,15:00,Premier League,Matchweek 4,Sat,Home,W,4.0,2.0,Crystal Palace,...,Darren England,Match Report,NaN,18.0,5.0,14.1,0.0,0.0,2023,Manchester-City
5,2022-08-31,19:30,Premier League,Matchweek 5,Wed,Home,W,6.0,0.0,Nott'ham Forest,...,Paul Tierney,Match Report,NaN,17.0,9.0,14.8,0.0,0.0,2023,Manchester-City
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1384,2021-05-02,19:15,Premier League,Matchweek 34,Sun,Away,L,0.0,4.0,Tottenham,...,Andre Marriner,Match Report,NaN,8.0,1.0,17.4,0.0,0.0,2021,Sheffield United
1385,2021-05-08,15:00,Premier League,Matchweek 35,Sat,Home,L,0.0,2.0,Crystal Palace,...,Simon Hooper,Match Report,NaN,7.0,0.0,11.4,0.0,0.0,2021,Sheffield United
1386,2021-05-16,19:00,Premier League,Matchweek 36,Sun,Away,W,1.0,0.0,Everton,...,Jonathan Moss,Match Report,NaN,10.0,3.0,17.0,0.0,0.0,2021,Sheffield United
1387,2021-05-19,18:00,Premier League,Matchweek 37,Wed,Away,L,0.0,1.0,Newcastle Utd,...,Robert Jones,Match Report,NaN,11.0,1.0,16.0,0.0,0.0,2021,Sheffield United


In [46]:
df.to_csv("data_final.csv")